# Segmenting and Clustering Neighborhoods in Toronto


## Part 1: Cleaning the data 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df= pd.read_html(url, header=0)[0]

#Renaming the column Postal Code
df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Removing the rows with 'Not assigned' Borough and reseting the indices:

In [7]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 3)

## Finding latitude and longitude data

In [9]:
#Loading the file containing the geaspacial data into a dataframe
url1='http://cocl.us/Geospatial_data'
df1=pd.read_csv(url1)

In [10]:
df1.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df1.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging the dataframe to obtain the coordinates of each neighborhood:

In [11]:
df2=pd.merge(df,df1, on="PostalCode",right_index=True, left_index=True)

In [97]:
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


## Part 2: Exploring Toronto and its neighborhoods

#### Number of neighborhoods in each Borough 

In [13]:
df2.groupby('Borough').count()

,PostalCode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


#### Creating a dataframe that contains all Borough containing the word 'Toronto' i.e. East Toronto, West Toronto, Downtown Toronto and Central Toronto:

In [50]:
toronto_data = df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,43.786947,-79.385975
5,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
6,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
7,M6G,Downtown Toronto,Christie,43.753259,-79.329656
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944


#### Geographical coordinates of Toronto:

In [99]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [100]:
# create map of Toronto city using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Extracting venues in selected neighborhoods using Foursquare credentials 

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ***
CLIENT_SECRET:***


#### Function that extracts the venues within a radius of 500 of each of the selected neighborhoods:

In [20]:
LIMIT = 100 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(761, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


#### Number of venues per neighborhood:

In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton, Parkdale Village, Exhibition Place",42,42,42,42,42,42
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",2,2,2,2,2,2
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",19,19,19,19,19,19
Central Bay Street,7,7,7,7,7,7
Christie,2,2,2,2,2,2
Church and Wellesley,7,7,7,7,7,7
"Commerce Court, Victoria Hotel",1,1,1,1,1,1
Davisville,5,5,5,5,5,5
Davisville North,60,60,60,60,60,60


#### Unique venues in the list:

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 199 uniques categories.


## Analysing each neighborhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,

In [26]:
toronto_onehot.shape

(761, 199)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Brockton, Parkdale Village, Exhibition Place",0.023810,0.000000,0.000000,0.000000,0.0,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.023810,0.023810,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.047619,0.000000,0.00,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.023810,0.000000,0.000000,0.023810,0.000000,0.071429,0.00,0.00,0.023810,0.000000,0.000000,0.0,0.023810,0.000000,0.02381,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.023810,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.023810,0.0000,0.02381,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.02381,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.0000,0.000000,0.023810,0.00,0.0,0.00,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.0000,0.00,0.023810,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.023810,0.00,0.00,0.000000,0

#### Confirming size

In [28]:
toronto_grouped.shape

(38, 199)

#### Here we notice that there are 38 neighborhood instead of 39 where venues have been listed, so let's find out which neighborhood does not have venues listed

In [43]:
toronto_data['Neighborhood'].isin(toronto_grouped['Neighborhood'])

0      True
1      True
2      True
3      True
4      True
5     False
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34     True
35     True
36     True
37     True
38     True
Name: Neighborhood, dtype: bool

#### So one of the neighborhood in the dataframe does not have any venues listed, we will eliminate it from the dataframe

In [51]:
toronto_data.drop(toronto_data.index[5], inplace=True)
toronto_data.shape

(38, 5)

#### Top 5 venues in each neighborhood:

In [52]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2            Gastropub  0.05
3              Brewery  0.05
4  American Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                             venue  freq
0                             Park   0.5
1                   Sandwich Place   0.5
2  Molecular Gastronomy Restaurant   0.0
3                   Lingerie Store   0.0
4                     Liquor Store   0.0


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Light Rail Station  0.11
1          Smoke Shop  0.05
2       Auto Workshop  0.05
3       Burrito Place  0.05
4             Brewery  0.05


----Central Bay Street----
            venue  freq
0   Grocery Store  0.14
1            Bank  0.14
2     Pizza Place  0.14
3     Coffee Shop  0.1

#### Puting it into a pandas dataframe

In [55]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating the new dataframe and display the top 10 venues for each neighborhood:

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Brewery,Bakery,American Restaurant,Gastropub,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
1,"Business reply mail Processing Centre, South C...",Park,Sandwich Place,Diner,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
2,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Comic Shop,Auto Workshop,Skate Park,Restaurant,Smoke Shop,Farmers Market,Fast Food Restaurant,Recording Studio,Spa
3,Central Bay Street,Discount Store,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Bank,Butcher,Women's Store,Falafel Restaurant,Ethiopian Restaurant
4,Christie,Park,Food & Drink Shop,Diner,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
5,Church and Wellesley,Pizza Place,Coffee Shop,Intersection,Chinese Restaurant,Sandwich Place,Discount Store,Women's Store,Diner,Ethiopian Restaurant,Electronics Store
6,"Commerce Court, Victoria Hotel",Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
7,Davisville,Park,Construction & Landscaping,Trail,Bakery,Deli / Bodega,Discount Store,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
8,Davisville North,Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Dessert Shop,Grocery Store,Park,Pizza Place
9,"Dufferin, Dovercourt Village",Park,Grocery Store,Shopping Mall,Bank,Hotel,Discount Store,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store


## Part 3: Clustering the Neighborhoods

#### Running k-means to cluster the neighborhood into 5 clusters:

In [88]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 0, 1, 2, 1, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 4, 0, 3, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

#### New dataframe that includes the cluster as well as the top 10 venues for each neighborhood:

In [90]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted.insert(0, 'Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

#toronto_merged['Labels']

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Mexican Restaurant,Rental Car Location,Breakfast Spot,Electronics Store,Intersection,Bank,Medical Center,Restaurant,Ethiopian Restaurant,Eastern European Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,0,Fried Chicken Joint,Thai Restaurant,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Lounge,Gas Station,Caribbean Restaurant,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0,Café,Skating Rink,College Stadium,General Entertainment,Women's Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Chinese Restaurant,Coffee Shop,Noodle House,Electronics Store,Fast Food Restaurant,Bubble Tea Shop,Sandwich Place,Discount Store,Bank,Supermarket
4,M4E,East Toronto,The Beaches,43.786947,-79.385975,0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Vizualising the neighborhood clusters:

In [91]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

#### Cluster 1

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Mexican Restaurant,Rental Car Location,Breakfast Spot,Electronics Store,Intersection,Bank,Medical Center,Restaurant,Ethiopian Restaurant,Eastern European Restaurant
1,Downtown Toronto,0,Fried Chicken Joint,Thai Restaurant,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Lounge,Gas Station,Caribbean Restaurant,Distribution Center
2,Downtown Toronto,0,Café,Skating Rink,College Stadium,General Entertainment,Women's Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant
3,Downtown Toronto,0,Chinese Restaurant,Coffee Shop,Noodle House,Electronics Store,Fast Food Restaurant,Bubble Tea Shop,Sandwich Place,Discount Store,Bank,Supermarket
4,East Toronto,0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
6,Downtown Toronto,0,Discount Store,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Bank,Butcher,Women's Store,Falafel Restaurant,Ethiopian Restaurant
8,Downtown Toronto,0,Park,Airport,Snack Place,Gift Shop,General Travel,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,West Toronto,0,Park,Grocery Store,Shopping Mall,Bank,Hotel,Discount Store,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
10,Downtown Toronto,0,Park,Curling Ice,Beer Store,Video Store,Skating Rink,Pharmacy,Athletics & Sports,Dance Studio,Dog Run,Doner Restaurant
11,West Toronto,0,Trail,Health Food Store,Pub,Asian Restaurant,Women's Store,Diner,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store


#### Cluster 2

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Park,Food & Drink Shop,Diner,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
16,Downtown Toronto,1,Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
24,Central Toronto,1,Park,Pool,Women's Store,Airport Food Court,Concert Hall,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
38,East Toronto,1,Park,Sandwich Place,Diner,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run


#### Cluster 3

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Downtown Toronto,2,Pizza Place,Furniture / Home Store,Women's Store,Dessert Shop,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run
37,Downtown Toronto,2,Pizza Place,Coffee Shop,Intersection,Chinese Restaurant,Sandwich Place,Discount Store,Women's Store,Diner,Ethiopian Restaurant,Electronics Store


#### Cluster 4

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,West Toronto,3,Grocery Store,Convenience Store,Pizza Place,Women's Store,Diner,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 5

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,4,Baseball Field,Home Service,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
36,Downtown Toronto,4,Baseball Field,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Doner Restaurant
